#### Auth for Spotify

In [ ]:
import pandas as pd
from auth import auth_with_credentials
from dotenv import load_dotenv
import os

load_dotenv()
SPOTIFY_ID=os.getenv('SPOTIFY_ID')
SPOTIFY_SECRET=os.getenv('SPOTIFY_SECRET')
redirect_uri = 'https://spotify.homeclean.site/callback'

In [ ]:
sp = auth_with_credentials(client_id=SPOTIFY_ID, client_secret=SPOTIFY_SECRET)

In [ ]:
from spotipy.oauth2 import SpotifyOAuth
import spotipy
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_ID,
        client_secret=SPOTIFY_SECRET,
        redirect_uri="https://spotify.homeclean.site/callback",
        scope = (
            "user-read-email "
            "user-library-read "
            "user-read-playback-state "
            "user-read-currently-playing "
            "user-modify-playback-state "
            "app-remote-control "
            "playlist-read-collaborative "
            "playlist-read-private"
        ),
        cache_handler=None
    )
)


In [ ]:
import spotipy
access_token='BQAXn3dt3V5VbfhV94uWv8H9IYajUQ4aFf2wSeNqpZbOR18fYs7rMdggy6x5HT1ZLKkVE9Spi-sB9h1ktcJum6hpa-JjfCJ8uJo_UCdBmDfGRoK98kCibgoFufhjQ7uzsbdMVi5znnNB860FRh51DzVXOPaps-nXSbjMs-Q3JEoFhVgwFYX94f6gLYCMaoFeuGZjWhnNNH-1vGFwC0g9dI0vsp8Y2oavOLM1EnZObbGBSJKyBbbogBW41hWYUMZmRZg8'
sp = spotipy.Spotify(auth=access_token)
print(sp.me())


In [ ]:
print(sp.current_playback())

In [ ]:
current_user_all_playlist = sp.current_user_playlists()

In [ ]:
import json
current_user_all_playlist['items'][0]

In [ ]:
play_list_name = current_user_all_playlist['items'][0]['name']
playlist_id = current_user_all_playlist['items'][0]['id']
owner_name = current_user_all_playlist['items'][0]['owner']['display_name']
owner_uri = current_user_all_playlist['items'][0]['owner']['uri']
is_public_playlist = current_user_all_playlist['items'][0]['public']
total = current_user_all_playlist['items'][0]['tracks']['total']
playlist_uri = current_user_all_playlist['items'][0]['uri']

In [ ]:
list_1 = []
list_1.append({
    'play_list_name':play_list_name,
    'playlist_id':play_list_name,
    'owner_name':owner_name,
    'owner_uri':owner_uri,
    'is_public_playlist':is_public_playlist,
    'total':total,
    'playlist_uri':playlist_uri
})

list_1

In [ ]:
list_1

In [ ]:
playlist_name = []
playlist_id = []
owner_name = []
owner_uri = []
is_public_playlist = []
total = []
playlist_uri = []

for i in range(0, 6):
    playlist_name.append(str(current_user_all_playlist['items'][i].get('name')))
    playlist_id.append(str(current_user_all_playlist['items'][i].get('id')))
    owner_name.append(str(current_user_all_playlist['items'][i]['owner'].get('display_name')))
    owner_uri.append(str(current_user_all_playlist['items'][i]['owner'].get('uri')))
    is_public_playlist.append(current_user_all_playlist['items'][i].get('public'))
    total.append(current_user_all_playlist['items'][i]['tracks'].get('total'))
    playlist_uri.append(str(current_user_all_playlist['items'][i].get('uri')))

df = pd.DataFrame({
    'playlist_name': playlist_name,
    'playlist_id': playlist_id,
    'owner_name': owner_name,
    'owner_uri': owner_uri,
    'is_public_playlist': is_public_playlist,
    'total': total,
    'playlist_uri': playlist_uri
})


In [ ]:
df = pd.json_normalize(current_user_all_playlist['items'])

In [ ]:
df.to_csv('raw_data/playlist.csv', index=False)

In [ ]:
import json

items = json.loads(json.dumps(current_user_all_playlist["items"]))

items

In [ ]:
playlist_name = []
playlist_id = []
owner_name = []
owner_uri = []
is_public_playlist = []
total = []
playlist_uri = []

for pl in items:
    playlist_name.append(pl["name"] )
    playlist_id.append(pl["id"] )
    owner_name.append(pl["owner"]["display_name"] )
    owner_uri.append(pl["owner"]["uri"].split(':')[-1] )
    is_public_playlist.append(pl["public"] )
    total.append(pl["tracks"]["total"] )
    playlist_uri.append(pl['uri'].split(':')[-1])

In [ ]:
playlist_id = df['playlist_uri'][0].split(':')[-1]

In [ ]:
playlist_id

In [ ]:
data = sp.playlist(playlist_id)

In [ ]:
len(data['tracks']['items'])

In [ ]:
sp.audio_analysis(track_id='5J90ah0ppUSev1uahqQiN6')

In [ ]:
sp.search(q='Vũ.')

#### Get raw data of some track

##### Get track infomation from Playlists on Spotify

In [30]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv
import os

load_dotenv()
SPOTIFY_ID=os.getenv('SPOTIFY_ID')
SPOTIFY_SECRET=os.getenv('SPOTIFY_SECRET')

In [3]:
import requests
base_url="https://acousticbrainz.org"

In [31]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_ID,
    client_secret=SPOTIFY_SECRET
))

track_id = "2P4OICZRVAQcYAV2JReRfj"  # ví dụ
track_info = sp.track(track_id)

track_name = track_info["name"]
artist_name = track_info["artists"][0]["name"]
print("Track:", track_name, "| Artist:", artist_name)

Track: Waiting For Love | Artist: Avicii


In [7]:
mb_search_url = "https://musicbrainz.org/ws/2/recording/"
params = {
    "query": f'track:"{track_name}" AND artist:"{artist_name}"',
    "fmt": "json"
}
resp = requests.get(mb_search_url, params=params, headers={"User-Agent": "Spotify2AcousticBrainz/1.0"})
mb_data = resp.json()

if "recordings" in mb_data and len(mb_data["recordings"]) > 0:
    mbid = mb_data["recordings"][0]["id"]
    print("MusicBrainz ID:", mbid)
else:
    print("❌ Không tìm thấy MBID")
    exit()

MusicBrainz ID: f8675903-5235-4e5c-a9be-89dd87341f3e


In [11]:
raw_data = sp.playlist_items(playlist_id='5oYN7C2h3aRI9OGNpXsd0F')

In [12]:
items = raw_data['items']

In [ ]:
track_id = items[0]['track']['id']
track_name = items[0]['track']['name']
artist_name = items[0]['track']['artists'][0]['name']
popularity = items[0]['track']['popularity']
duration_ms = items[0]['track']['duration_ms']

'Avicii'

In [16]:
tracks_id = []
tracks_name = []
artist_name = []
popularity = []
duration_ms = []

for i, track in enumerate(items):
    tracks_id.append(track['track']['id'])
    tracks_name.append(track['track']['name'])
    artist_name.append(track['track']['artists'][0]['name'])
    popularity.append(track['track']['popularity'])
    duration_ms.append(track['track']['duration_ms'])


In [ ]:
import pandas as pd
from datetime import datetime, timedelta


df_tracks = pd.DataFrame({
    "track_id": tracks_id,
    "track_name": tracks_name,
    "artist_name": artist_name,
    "popularity": popularity,
    "duration_ms": duration_ms
})

df_tracks.to_csv(
    f"raw_data/tracks_name_{datetime.today().strftime('%Y%m%d')}.csv",
    index=False
)

In [20]:
import requests
from bs4 import BeautifulSoup
data = requests.get('https://www.pvoil.com.vn/tin-gia-xang-dau')

if data.status_code == 200:
    print('ok')

ok


In [ ]:
data.content
soup = BeautifulSoup(data.content, 'lxml')

In [23]:
import pandas as pd
from datetime import datetime
from io import StringIO

table = soup.find('table')

# Bọc HTML string bằng StringIO
df = pd.read_html(StringIO(str(table)))[0]

# Giữ 2 cột cần thiết
df = df.iloc[:, [1, 3]]
df.columns = ["Categories", "Price(VND/l)"]
df['updated_date'] = datetime.today()
df["Price(VND/l)"] = (
    df["Price(VND/l)"]
    .astype(str)
    .str.replace("đ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.strip()
    .astype(int)
)
df.to_csv('raw_data/fuel_price.csv', index=False, mode='a')

##### Get audio feature

In [33]:
df_1 = pd.read_csv('raw_data/tracks_name_20250904.csv')

In [ ]:
import requests
base_url='https://musicbrainz.org/ws/2/recording'

In [32]:
sp.user("diatimbaaang")

{'display_name': 'd!yeah',
 'external_urls': {'spotify': 'https://open.spotify.com/user/diatimbaaang'},
 'followers': {'href': None, 'total': 39},
 'href': 'https://api.spotify.com/v1/users/diatimbaaang',
 'id': 'diatimbaaang',
 'images': [{'url': 'https://i.scdn.co/image/ab6775700000ee85a96f86f69693e73dd5c2fcfc',
   'height': 300,
   'width': 300},
  {'url': 'https://i.scdn.co/image/ab67757000003b82a96f86f69693e73dd5c2fcfc',
   'height': 64,
   'width': 64}],
 'type': 'user',
 'uri': 'spotify:user:diatimbaaang'}

In [35]:
sp.user_playlists('diatimbaaang')

{'href': 'https://api.spotify.com/v1/users/diatimbaaang/playlists?offset=0&limit=50',
 'limit': 50,
 'next': None,
 'offset': 0,
 'previous': None,
 'total': 6,
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/4xEz5fJoVPomSE7wz3oC1i'},
   'href': 'https://api.spotify.com/v1/playlists/4xEz5fJoVPomSE7wz3oC1i',
   'id': '4xEz5fJoVPomSE7wz3oC1i',
   'images': [{'height': None,
     'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da8451276bdbb3cb92d83ec67adf',
     'width': None}],
   'name': 'shrek',
   'owner': {'display_name': 'd!yeah',
    'external_urls': {'spotify': 'https://open.spotify.com/user/diatimbaaang'},
    'href': 'https://api.spotify.com/v1/users/diatimbaaang',
    'id': 'diatimbaaang',
    'type': 'user',
    'uri': 'spotify:user:diatimbaaang'},
   'primary_color': None,
   'public': True,
   'snapshot_id': 'AAAAFTC1bhS6dZ4ED0Zeval3EEgDmg4P',
   'tracks': {'href': 'https://api.spotify

In [ ]:
import http.client

conn = http.client.HTTPSConnection("therundown-therundown-v1.p.rapidapi.com")

headers = {

}

conn.request("GET", "/sports", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"sports":[{"sport_id":1,"sport_name":"NCAA Football"},{"sport_id":2,"sport_name":"NFL"},{"sport_id":3,"sport_name":"MLB"},{"sport_id":4,"sport_name":"NBA"},{"sport_id":5,"sport_name":"NCAA Men's Basketball"},{"sport_id":6,"sport_name":"NHL"},{"sport_id":7,"sport_name":"UFC/MMA"},{"sport_id":8,"sport_name":"WNBA"},{"sport_id":10,"sport_name":"MLS"},{"sport_id":11,"sport_name":"EPL"},{"sport_id":12,"sport_name":"FRA1"},{"sport_id":13,"sport_name":"GER1"},{"sport_id":14,"sport_name":"ESP1"},{"sport_id":15,"sport_name":"ITA1"},{"sport_id":16,"sport_name":"UEFACHAMP"},{"sport_id":17,"sport_name":"UEFAEURO"},{"sport_id":18,"sport_name":"FIFA"},{"sport_id":19,"sport_name":"JPN1"},{"sport_id":20,"sport_name":"IPL"},{"sport_id":21,"sport_name":"T20"},{"sport_id":22,"sport_name":"Politics"},{"sport_id":23,"sport_name":"NBA Preseason"},{"sport_id":24,"sport_name":"NBA Playoffs"},{"sport_id":25,"sport_name":"NFL Preseason"},{"sport_id":26,"sport_name":"NFL Playoffs"},{"sport_id":27,"sport_name":"

In [41]:
import json
sports_raw = data.decode("utf-8")
sports_json = json.loads(sports_raw)  # Dạng dict với key 'sports'
sports_list = sports_json["sports"]   # Lấy list sports

In [43]:
df = pd.DataFrame(sports_list)

In [45]:
df.to_csv("raw_data/TheRundown_sports.csv", index=False)

In [ ]:
import http.client

conn = http.client.HTTPSConnection("therundown-therundown-v1.p.rapidapi.com")

headers = {

}

conn.request("GET", "/sports/4/teams", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"teams":[{"team_id":2776,"name":"Adelaide","mascot":"Adelaide 36ers","abbreviation":"ADE","record":"5-5-7"},{"team_id":11,"name":"Atlanta","mascot":"Hawks","abbreviation":"ATL","record":"40-42","conference":{"conference_id":39,"sport_id":4,"name":"Eastern Conference"},"division":{"division_id":17,"conference_id":39,"sport_id":4,"name":"Southeast"}},{"team_id":1,"name":"Boston","mascot":"Celtics","abbreviation":"BOS","record":"61-21","conference":{"conference_id":39,"sport_id":4,"name":"Eastern Conference"},"division":{"division_id":15,"conference_id":39,"sport_id":4,"name":"Atlantic"}},{"team_id":2,"name":"Brooklyn","mascot":"Nets","abbreviation":"BKN","record":"26-56","conference":{"conference_id":39,"sport_id":4,"name":"Eastern Conference"},"division":{"division_id":15,"conference_id":39,"sport_id":4,"name":"Atlantic"}},{"team_id":10443,"name":"Cairns","mascot":"Tiapans","abbreviation":"CNS"},{"team_id":12,"name":"Charlotte","mascot":"Hornets","abbreviation":"CHA","record":"19-63","

In [ ]:
nba_team_raw = data.decode("utf-8")
nba_team_raw_json = json.loads(nba_team_raw)  # Dạng dict với key 'sports'
nba_team_list = nba_team_raw_json["teams"]   # Lấy list sports
nba_team_list

In [53]:
df = pd.json_normalize(nba_team_list)
df

,team_id,name,mascot,abbreviation,record,conference.conference_id,conference.sport_id,conference.name,division.division_id,division.conference_id,division.sport_id,division.name
0,2776,Adelaide,Adelaide 36ers,ADE,5-5-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,Atlanta,Hawks,ATL,40-42,39.0,4.0,Eastern Conference,17.0,39.0,4.0,Southeast
2,1,Boston,Celtics,BOS,61-21,39.0,4.0,Eastern Conference,15.0,39.0,4.0,Atlantic
3,2,Brooklyn,Nets,BKN,26-56,39.0,4.0,Eastern Conference,15.0,39.0,4.0,Atlantic
4,10443,Cairns,Tiapans,CNS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
56,20,Utah,Jazz,UTAH,17-65,757.0,4.0,Western Conference,18.0,757.0,4.0,Northwest
57,3530,Utah,Jazz Blue,UTAH,5-5-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,3531,Utah,Jazz White,UTAH,5-5-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,15,Washington,Wizards,WSH,18-64,39.0,4.0,Eastern Conference,17.0,39.0,4.0,Southeast


In [55]:
df = df.dropna()

In [57]:
df.to_csv("raw_data/nba_teams.csv", index=False)